# Mendeteksi dan Mengurangi Ketidakwajaran dalam Model

Model pembelajaran mesin dapat menggabungkan bias yang tidak disengaja, yang dapat menyebabkan masalah dengan *kewajaran*. Misalnya, model yang memprediksi kemungkinan diabetes mungkin bekerja dengan baik untuk beberapa kelompok usia, tetapi tidak untuk beberapa kelompok yang lain - mengharuskan sebagian pasien untuk mengikuti pengujian yang tidak perlu, atau menghapus beberapa pengujian yang akan mengonfirmasi diagnosis diabetes.

Dalam buku catatan ini, Anda akan menggunakan paket **Fairlearn** untuk menganalisis model dan mengeksplorasi perbedaan performa prediksi untuk subset pasien yang berbeda berdasarkan usia.

> **Catatan**: Integrasi dengan paket Fairlearn saat ini sedang dalam pratinjau. Anda mungkin mengalami beberapa kesalahan tak terduga.

## Penting - Pertimbangan untuk kewajaran

> Notebook ini dirancang sebagai latihan praktis untuk membantu Anda menjelajahi paket Fairlearn dan integrasinya dengan Azure Machine Learning. Namun, ada banyak pertimbangan yang harus didiskusikan oleh organisasi atau tim ilmu data terkait dengan kewajaran sebelum menggunakan alat tersebut. Kewajaran adalah tantangan *sosioteknik* kompleks yang lebih dari sekadar menjalankan alat untuk menganalisis model.
>
> Microsoft Research telah bersama-sama mengembangkan [daftar periksa kewajaran](https://www.microsoft.com/en-us/research/publication/co-designing-checklists-to-understand-organizational-challenges-and-opportunities-around-fairness-in-ai/) yang memberikan titik awal yang bagus untuk diskusi penting yang perlu dilakukan sebelum satu baris kode ditulis.

## Menginstal SDK yang diperlukan

Untuk menggunakan paket Fairlearn dengan Azure Machine Learning, Anda memerlukan paket Azure Machine Learning dan Python Fairlearn, jadi jalankan sel berikut untuk memverifikasi bahwa paket **azureml-contrib-fairness** telah diinstal. 

In [ ]:
pip show azureml-contrib-fairness

Anda juga memerlukan paket **fairlearn** itu sendiri, dan paket **raiwidgets** (yang digunakan oleh Fairlearn untuk memvisualisasikan dasbor). Jalankan sel berikut untuk menginstalnya.

In [ ]:
pip install --upgrade fairlearn==0.7.0 raiwidgets

## Melatih model

Anda akan mulai dengan melatih model klasifikasi untuk memprediksi kemungkinan diabetes. Selain membagi data menjadi kumpulan fitur serta label pelatihan dan pengujian, Anda akan mengekstrak fitur *sensitif* yang digunakan untuk menentukan subpopulasi data yang kewajarannya ingin Anda bandingkan. Dalam hal ini, Anda akan menggunakan kolom **Usia** untuk menentukan dua kategori pasien: pasien yang berusia di atas 50 tahun, dan pasien yang berusia 50 tahun atau lebih muda.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
X, y = data[features].values, data['Diabetic'].values

# Get sensitive features
S = data[['Age']].astype(int)
# Change value to represent age groups
S['Age'] = np.where(S.Age > 50, 'Over 50', '50 or younger')

# Split data into training set and test set
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.20, random_state=0, stratify=y)

# Train a classification model
print("Training model...")
diabetes_model = DecisionTreeClassifier().fit(X_train, y_train)

print("Model trained.")

Sekarang setelah melatih model, Anda dapat menggunakan paket Fairlearn guna membandingkan perilakunya untuk nilai fitur sensitif yang berbeda. Dalam hal ini, Anda akan:

- Menggunakan fungsi **selection_rate** fairlearn untuk mengembalikan tingkat pemilihan (persentase prediksi positif) untuk keseluruhan populasi.
- Menggunakan fungsi metrik **scikit-learn** untuk menghitung akurasi, ingatan, dan metrik presisi secara keseluruhan.
- Menggunakan **MetricFrame** untuk menghitung tingkat pemilihan, akurasi, ingatan, dan presisi untuk setiap kelompok usia di fitur sensitif **Usia**. Perhatikan bahwa campuran fungsi metrik **fairlearn** dan **scikit-learn** digunakan untuk menghitung nilai performa.

In [ ]:
from fairlearn.metrics import selection_rate, MetricFrame
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Get predictions for the witheld test data
y_hat = diabetes_model.predict(X_test)

# Get overall metrics
print("Overall Metrics:")
# Get selection rate from fairlearn
overall_selection_rate = selection_rate(y_test, y_hat) # Get selection rate from fairlearn
print("\tSelection Rate:", overall_selection_rate)
# Get standard metrics from scikit-learn
overall_accuracy = accuracy_score(y_test, y_hat)
print("\tAccuracy:", overall_accuracy)
overall_recall = recall_score(y_test, y_hat)
print("\tRecall:", overall_recall)
overall_precision = precision_score(y_test, y_hat)
print("\tPrecision:", overall_precision)

# Get metrics by sensitive group from fairlearn
print('\nMetrics by Group:')
metrics = {'selection_rate': selection_rate,
           'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score}

group_metrics = MetricFrame(metrics=metrics,
                             y_true=y_test,
                             y_pred=y_hat,
                             sensitive_features=S_test['Age'])

print(group_metrics.by_group)

Dari metrik ini, Anda harus dapat membedakan bahwa sebagian besar pasien yang lebih tua diprediksi menderita diabetes. *Akurasi* harus kurang lebih sama untuk kedua kelompok, tetapi pemeriksaan lebih jauh terhadap *presisi* dan *pengenalan* menunjukkan beberapa perbedaan dalam seberapa baik model memprediksi setiap kelompok usia.

Dalam skenario ini, pertimbangkan *pengenalan*. Metrik ini menunjukkan proporsi kasus positif yang diidentifikasi dengan benar oleh model. Dengan kata lain, dari semua pasien yang benar-benar diabetes, berapa banyak yang ditemukan model? Model ini melakukan pekerjaan yang lebih baik untuk pasien dalam kelompok usia yang lebih tua daripada pasien yang lebih muda.

Seringkali lebih mudah untuk membandingkan metrik secara visual. Untuk melakukan ini, Anda akan menggunakan dasbor kewajaran Fairlearn:

1. Jalankan sel di bawah ini untuk membuat dasbor dari model yang Anda buat sebelumnya.
2. Saat widget ditampilkan, gunakan tautan **Mulai** untuk mulai mengonfigurasi visualisasi Anda.
3. Pilih fitur sensitif yang ingin Anda bandingkan (dalam hal ini, hanya ada satu: **Usia**).
4. Pilih metrik performa model yang ingin Anda bandingkan (dalam hal ini, ini adalah model klasifikasi biner sehingga opsinya adalah *Akurasi*, *Akurasi seimbang*, *Presisi*, dan *Pengenalan*). Mulailah dengan **Pengenalan**.
5. Pilih jenis perbandingan kewajaran yang ingin Anda lihat. Mulailah dengan **Perbedaan paritas demografis**.
6. Lihat bagan dasbor, yang menunjukkan:
    - **Tingkat seleksi** - Perbandingan jumlah kasus positif per subpopulasi.
    - **Tingkat positif palsu dan negatif palsu** - bagaimana metrik performa yang dipilih dibandingkan untuk subpopulasi, termasuk *prediksi rendah* (negatif palsu) dan *prediksi berlebihan* (positif palsu).
7. Edit konfigurasi untuk membandingkan prediksi berdasarkan metrik performa dan kewajaran yang berbeda.

In [ ]:
from raiwidgets import FairnessDashboard

# View this model in Fairlearn's fairness dashboard, and see the disparities which appear:
FairnessDashboard(sensitive_features=S_test,
                   y_true=y_test,
                   y_pred={"diabetes_model": diabetes_model.predict(X_test)})

Hasilnya menunjukkan tingkat seleksi yang jauh lebih tinggi untuk pasien berusia di atas 50 tahun daripada pasien yang lebih muda. Namun, pada kenyataannya, usia merupakan faktor utama dalam diabetes, jadi Anda akan mengharapkan lebih banyak kasus positif di antara pasien yang lebih tua.

Jika kita mendasarkan performa model pada *akurasi* (dengan kata lain, persentase prediksi model menjadi benar), maka tampaknya bekerja kurang lebih sama untuk kedua subpopulasi. Namun, berdasarkan metrik *presisi* dan *pengenalan*, model ini cenderung berperforma lebih baik untuk pasien yang berusia di atas 50 tahun.

Mari kita lihat apa yang terjadi jika kita mengecualikan fitur **Usia** saat melatih model.

In [ ]:
# Separate features and labels
ageless = features.copy()
ageless.remove('Age')
X2, y2 = data[ageless].values, data['Diabetic'].values

# Split data into training set and test set
X_train2, X_test2, y_train2, y_test2, S_train2, S_test2 = train_test_split(X2, y2, S, test_size=0.20, random_state=0, stratify=y2)

# Train a classification model
print("Training model...")
ageless_model = DecisionTreeClassifier().fit(X_train2, y_train2)
print("Model trained.")

# View this model in Fairlearn's fairness dashboard, and see the disparities which appear:
FairnessDashboard(sensitive_features=S_test2,
                   y_true=y_test2,
                   y_pred={"ageless_diabetes_model": ageless_model.predict(X_test2)})

Jelajahi model di dasbor.

Saat Anda meninjau *pengenalan*, perhatikan bahwa disparitas telah berkurang, tetapi keseluruhan pengenalan juga berkurang karena model sekarang secara signifikan memperkirakan kasus positif untuk pasien yang lebih tua. Meskipun **Usia** bukan fitur yang digunakan dalam pelatihan, model tersebut masih menunjukkan beberapa perbedaan dalam seberapa baik prediksinya untuk pasien yang lebih tua dan lebih muda.

Dalam skenario ini, menghapus fitur **Umur** sedikit mengurangi perbedaan dalam *pengenalan*, tetapi meningkatkan perbedaan dalam *presisi* dan *akurasi*. Hal ini menggarisbawahi salah satu kesulitan utama dalam menerapkan kewajaran pada model pembelajaran mesin - Anda harus benar-benar memahami apa yang dimaksud dengan *kewajaran* dalam konteks tertentu, dan mengoptimalkannya.

## Mendaftarkan model dan mengunggah data dasbor ke ruang kerja Anda

Anda telah melatih model dan meninjau dasbor secara lokal di notebook ini; tetapi mungkin berguna untuk mendaftarkan model di ruang kerja Azure Machine Learning Anda dan membuat eksperimen untuk merekam data dasbor sehingga Anda dapat melacak dan membagikan analisis kewajaran.

Mari kita mulai dengan mendaftarkan model asli (yang menyertakan **Usia** sebagai fitur).

> **Catatan**: Jika Anda belum membuat sesi yang terautentikasi dengan langganan Azure, Anda akan diminta untuk mengautentikasi dengan mengklik tautan, memasukkan kode autentikasi, dan masuk ke Azure.

In [ ]:
from azureml.core import Workspace, Experiment, Model
import joblib
import os

# Load the Azure ML workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=diabetes_model, filename=model_file)

# Register the model
print('Registering model...')
registered_model = Model.register(model_path=model_file,
                                  model_name='diabetes_classifier',
                                  workspace=ws)
model_id= registered_model.id


print('Model registered.', model_id)

Sekarang Anda dapat menggunakan paket FairLearn untuk membuat kumpulan metrik grup klasifikasi biner untuk satu atau beberapa model, dan menggunakan eksperimen Azure Machine Learning untuk mengunggah metrik.

> **Catatan**: Ini mungkin memakan waktu cukup lama, dan dapat mengakibatkan beberapa pesan peringatan (yang dapat Anda abaikan). Ketika eksperimen selesai, data dasbor akan diunduh dan ditampilkan untuk memverifikasi bahwa data tersebut berhasil diunggah.

In [ ]:
from fairlearn.metrics._group_metric_set import _create_group_metric_set
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

#  Create a dictionary of model(s) you want to assess for fairness 
sf = { 'Age': S_test.Age}
ys_pred = { model_id:diabetes_model.predict(X_test) }
dash_dict = _create_group_metric_set(y_true=y_test,
                                    predictions=ys_pred,
                                    sensitive_features=sf,
                                    prediction_type='binary_classification')

exp = Experiment(ws, 'mslearn-diabetes-fairness')
print(exp)

run = exp.start_logging()

# Upload the dashboard to Azure Machine Learning
try:
    dashboard_title = "Fairness insights of Diabetes Classifier"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    # To test the dashboard, you can download it
    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
    print(downloaded_dict)
finally:
    run.complete()

Kode sebelumnya mengunduh metrik yang dihasilkan dalam eksperimen hanya untuk mengonfirmasi bahwa kode tersebut berhasil diselesaikan. Manfaat nyata dari mengunggah metrik ke eksperimen adalah Anda sekarang dapat melihat dasbor FairLearn di studio Azure Machine Learning.

Jalankan sel di bawah untuk melihat detail eksperimen, dan klik tautan **Lihat detail Eksekusi** di widget untuk melihat eksekusi di studio Azure Machine Learning. Kemudian lihat tab **Kewajaran** dari eksperimen yang dijalankan untuk melihat dasbor ID kewajaran yang ditetapkan ke metrik yang Anda unggah, yang berperilaku sama seperti widget yang Anda lihat sebelumnya di buku catatan ini.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Anda juga dapat menemukan dasbor kewajaran dengan memilih model di halaman **Model** di studio Azure Machine Learning dan meninjau tab **Fairness**-nya. Ini memungkinkan organisasi Anda untuk mempertahankan log analisis kewajaran untuk model yang Anda latih dan daftarkan.

## Mengurangi ketidakwajaran dalam model

Sekarang setelah Anda menganalisis model untuk kewajaran, Anda dapat menggunakan salah satu teknik *mitigasi* yang didukung oleh paket FairLearn untuk menemukan model yang menyeimbangkan performa prediktif dan kewajaran.

Dalam latihan ini, Anda akan menggunakan fitur **GridSearch**, yang melatih beberapa model dalam upaya meminimalkan perbedaan performa prediktif untuk fitur sensitif dalam kumpulan data (dalam hal ini, kelompok usia). Anda akan mengoptimalkan model dengan menerapkan batasan paritas **EqualizedOdds**, yang mencoba memastikan bahwa model yang menunjukkan tingkat positif benar dan salah serupa untuk setiap pengelompokan fitur sensitif. 

> *Ini mungkin butuh waktu untuk dijalankan*

In [ ]:
from fairlearn.reductions import GridSearch, EqualizedOdds
import joblib
import os

print('Finding mitigated models...')

# Train multiple models
sweep = GridSearch(DecisionTreeClassifier(),
                   constraints=EqualizedOdds(),
                   grid_size=20)

sweep.fit(X_train, y_train, sensitive_features=S_train.Age)
models = sweep.predictors_

# Save the models and get predictions from them (plus the original unmitigated one for comparison)
model_dir = 'mitigated_models'
os.makedirs(model_dir, exist_ok=True)
model_name = 'diabetes_unmitigated'
print(model_name)
joblib.dump(value=diabetes_model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
predictions = {model_name: diabetes_model.predict(X_test)}
i = 0
for model in models:
    i += 1
    model_name = 'diabetes_mitigated_{0}'.format(i)
    print(model_name)
    joblib.dump(value=model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
    predictions[model_name] = model.predict(X_test)


Sekarang Anda dapat menggunakan dasbor FairLearn untuk membandingkan model yang dimitigasi:

Jalankan sel berikut lalu gunakan wizard untuk memvisualisasikan **Umur** dengan **Pengenalan**.

In [ ]:
FairnessDashboard(sensitive_features=S_test,
                   y_true=y_test,
                   y_pred=predictions)

Model ditampilkan pada plot pencar. Anda dapat membandingkan model dengan mengukur perbedaan dalam prediksi (dengan kata lain, tingkat pemilihan) atau perbedaan dalam metrik performa yang dipilih (dalam hal ini, *pengenalan*). Dalam skenario ini, kami memperkirakan perbedaan dalam tingkat seleksi (karena kami tahu bahwa usia *adalah* faktor diabetes, dengan lebih banyak kasus positif pada kelompok usia yang lebih tua). Yang kami minati adalah perbedaan dalam performa prediktif, jadi pilih opsi untuk mengukur **Disparitas dalam ingatan**.

Diagram menunjukkan kluster model dengan metrik *pengenalan* keseluruhan pada sumbu X, dan disparitas pengenalan pada sumbu Y. Oleh karena itu, model yang ideal (dengan daya pengenalan tinggi dan disparitas rendah) akan berada di sudut kanan bawah plot. Anda dapat memilih keseimbangan yang tepat antara performa prediktif dan kewajaran untuk kebutuhan khusus Anda, dan memilih model yang sesuai untuk melihat detailnya.

Poin penting yang perlu diperkuat adalah bahwa menerapkan mitigasi kewajaran pada model adalah pertukaran antara performa prediktif keseluruhan dan perbedaan di seluruh grup fitur sensitif - umumnya Anda harus mengorbankan beberapa performa prediktif keseluruhan untuk memastikan bahwa model memprediksi secara adil untuk semua segmen populasi.

> **Catatan**: Melihat metrik *presisi* dapat mengakibatkan peringatan bahwa presisi diatur ke 0,0 karena tidak ada sampel yang diprediksi - Anda dapat mengabaikannya.

## Unggah metrik dasbor mitigasi ke Azure Machine Learning

Seperti sebelumnya, Anda mungkin ingin melacak eksperimen mitigasi Anda. Untuk melakukan ini, Anda dapat:

1. Mendaftarkan model yang ditemukan oleh proses GridSearch.
2. Menghitung metrik performa dan perbedaan untuk model.
3. Mengunggah metrik dalam eksperimen Azure Machine Learning.

In [ ]:
# Register the models
registered_model_predictions = dict()
for model_name, prediction_data in predictions.items():
    model_file = os.path.join(model_dir, model_name + ".pkl")
    registered_model = Model.register(model_path=model_file,
                                      model_name=model_name,
                                      workspace=ws)
    registered_model_predictions[registered_model.id] = prediction_data

#  Create a group metric set for binary classification based on the Age feature for all of the models
sf = { 'Age': S_test.Age}
dash_dict = _create_group_metric_set(y_true=y_test,
                                     predictions=registered_model_predictions,
                                     sensitive_features=sf,
                                     prediction_type='binary_classification')

exp = Experiment(ws, "mslearn-diabetes-fairness")
print(exp)

run = exp.start_logging()
RunDetails(run).show()

# Upload the dashboard to Azure Machine Learning
try:
    dashboard_title = "Fairness Comparison of Diabetes Models"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))
finally:
    run.complete()

> **Catatan**: Peringatan bahwa presisi sedang diatur ke 0,0 karena tidak ada sampel yang diprediksi dapat ditampilkan - Anda dapat mengabaikannya.


Saat eksperimen selesai berjalan, klik tautan **Lihat detail Eksekusi** di widget untuk melihat eksekusi di studio Azure Machine Learning (Anda mungkin perlu menggulir melewati output awal untuk melihat widget), dan melihat dasbor FairLearn pada tab **kewajaran**.